# Image classification with 'image_dataset_from_directory' method

# Creating train/val/test datasets using split-folders package

The split-folders package allow us to create 3 folders of train/val/test images

The input folder should have the following format (Gregoire's function output):

input/

    class1/
        img1.jpg
        img2.jpg
        ...
    class2/
        imgWhatever.jpg
        ...
    ...
    
    
And we get this

output/

    train/
        class1/
            img1.jpg
            ...
        class2/
            imga.jpg
            ...
            
    val/
    
        class1/
        
            img2.jpg
            ...
        class2/
            imgb.jpg
            ...
    test/
        class1/
            img3.jpg
            ...
        class2/
            imgc.jpg
            ...

In [1]:
TRAIN_RATIO = 0.8
VAL_RATIO = 0.1
TEST_RATIO = 0.1

assert TRAIN_RATIO + VAL_RATIO + TEST_RATIO == 1
assert TRAIN_RATIO != 0
assert VAL_RATIO != 0
assert TEST_RATIO != 0

In [2]:
import splitfolders
!rm -rf "/home/jupyter/wikiart/train_val_test_True_1440"
splitfolders.ratio("/home/jupyter/wikiart/wikiart-movement-genre_True-class_8-merge_mov-1-n_1440_max", 
                   output="/home/jupyter/wikiart/train_val_test_True_1440",
                   seed=1337, ratio=(TRAIN_RATIO, VAL_RATIO, TEST_RATIO), 
                   group_prefix=None);

Copying files: 11520 files [00:03, 2940.17 files/s]


# Deep learning workflow

## Imports

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import os
import datetime
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers, models, applications

# Note : we are using TensorFlow Core v2.5.0, in TensorFlow Core v2.6.0 all the data augmentaation layers are part of tf.keras.layers
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation, RandomZoom
from tensorflow.keras import Input, Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard, ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adamax

## Global variables

In [4]:
USERNAME = 'pablo'
MODEL = 'VGG16'

MAIN_PATH = '/home/jupyter/' 
DATASETS_FOLDER = 'wikiart/train_val_test_True_1440/'

TRAIN_DIR = MAIN_PATH + DATASETS_FOLDER + 'train'
VAL_DIR = MAIN_PATH + DATASETS_FOLDER + 'val'
TEST_DIR = MAIN_PATH + DATASETS_FOLDER + 'test'

In [5]:
BATCH_SIZE = 128
EPOCHS = 1000
IMG_HEIGHT = 224
IMG_WIDTH = 224
NUM_CLASSES = 8

## Datasets setup

### Train dataset

In [6]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=TRAIN_DIR,
    labels='inferred',
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    label_mode='categorical',
    shuffle=True)

assert len(train_ds.class_names) == NUM_CLASSES

Found 9216 files belonging to 8 classes.


### Validation dataset

In [7]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=VAL_DIR,
    labels='inferred',
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    label_mode='categorical',
    batch_size=BATCH_SIZE)

assert len(val_ds.class_names) == NUM_CLASSES

Found 1152 files belonging to 8 classes.


### Test dataset

In [8]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=TEST_DIR,
    labels='inferred',
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    label_mode='categorical',
    batch_size=BATCH_SIZE)

assert len(test_ds.class_names) == NUM_CLASSES

Found 1152 files belonging to 8 classes.


In [9]:
total_images_count = (int(len(list(train_ds)))+int(len(list(val_ds)))+int(len(list(test_ds))))*BATCH_SIZE
# total_images_count = 33011 + 4123 + 4134
total_images_count

11520

## Dataset optimization for performance

In [10]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(int(total_images_count/10)).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

## Model : transfer learning with VGG16

### VGG16 layers

In [11]:
layer_model = applications.VGG16(
    include_top=False, 
    weights='imagenet', 
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), 
    classes=NUM_CLASSES)

In [12]:
layer_model.layers

In [13]:
layer_model.trainable = False

In [14]:
layer_model.layers[-2:]

In [15]:
for layer in layer_model.layers[-2:]:
    layer.trainable = True
    

In [16]:
trainable_layer_count = 0

for i in range(len(layer_model.layers)):
    if layer_model.layers[i].trainable:
        trainable_layer_count += 1
        
trainable_layer_count

2

### Data augmentation layers

In [17]:
data_augmentation_layers = models.Sequential([
    RandomFlip("horizontal", input_shape=(224, 224,3)),
    RandomRotation(0.3),
    RandomZoom(0.3)])

### Model setup

In [18]:
tf.keras.backend.clear_session() # Clear the layers name (in case you run multiple time the cell)

inputs = Input(shape=(224, 224, 3))

x = data_augmentation_layers(inputs) # Are not applied to validation and test dataset (made inactive, tensorflow handle it)
x = applications.vgg16.preprocess_input(x) # Does the rescaling
x = layer_model(x) 
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x) 

outputs = layers.Dense(NUM_CLASSES, activation='softmax', name='classification_layer')(x)

model = Model(inputs, outputs)

### Callbacks

#### EarlyStopping

In [19]:
es = EarlyStopping(monitor='val_loss', patience=20, mode='min', restore_best_weights=True)

#### ReduceLROnPlateau

In [26]:
rlrp = ReduceLROnPlateau(monitor='val_loss', factor=0.4, patience=3, min_lr=1e-8)

#### ModelCheckpoint

In [21]:
checkpoint_dir = f"{MAIN_PATH}logs/{USERNAME}/{MODEL}/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + f"-unfreeze_{trainable_layer_count}"
mcp = ModelCheckpoint(
    filepath=checkpoint_dir,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_freq=10,
    save_best_only=True)

In [22]:
%load_ext tensorboard
# !rm -rf ./logs/ResNet50
recorded_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S") 
log_dir = f"{MAIN_PATH}logs/{USERNAME}/{MODEL}/" + \
    recorded_time + \
    f"-images_{total_images_count}" + \
    f"-unfreeze_{trainable_layer_count}" + \
    f"-batch_{BATCH_SIZE}"
tsboard = TensorBoard(log_dir=log_dir) # voir https public

### Model compilation

In [25]:
model.compile(optimizer=Adamax(learning_rate=0.001), 
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

### Model fitting

In [24]:
history = model.fit(
    train_ds, 
    epochs=EPOCHS, 
    validation_data=val_ds, 
    callbacks=[es, rlrp, tsboard], 
    use_multiprocessing=True)

model.save(f"{MAIN_PATH}models/{USERNAME}/{MODEL}/" + \
    recorded_time + \
    f"-images_{total_images_count}" + \
    f"-unfreeze_{trainable_layer_count}" + \
    f"-batch_{BATCH_SIZE}")

Epoch 1/1000
72/72 [==============================] - 86s 616ms/step - loss: 4.3145 - accuracy: 0.3708 - val_loss: 1.9313 - val_accuracy: 0.5347
Epoch 2/1000
72/72 [==============================] - 43s 594ms/step - loss: 2.3179 - accuracy: 0.4596 - val_loss: 1.6383 - val_accuracy: 0.5391
Epoch 3/1000
72/72 [==============================] - 43s 592ms/step - loss: 1.8028 - accuracy: 0.4869 - val_loss: 1.3055 - val_accuracy: 0.5582
Epoch 4/1000
72/72 [==============================] - 43s 596ms/step - loss: 1.5749 - accuracy: 0.4976 - val_loss: 1.2923 - val_accuracy: 0.5747
Epoch 5/1000
72/72 [==============================] - 43s 595ms/step - loss: 1.5137 - accuracy: 0.4976 - val_loss: 1.2581 - val_accuracy: 0.5530
Epoch 6/1000
72/72 [==============================] - 43s 594ms/step - loss: 1.5201 - accuracy: 0.5005 - val_loss: 1.2272 - val_accuracy: 0.5521
Epoch 7/1000
72/72 [==============================] - 43s 594ms/step - loss: 1.4973 - accuracy: 0.4916 - val_loss: 1.2226 - val_ac

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = history.epoch

fig = plt.figure(figsize=(16, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()
fig.savefig(f"/home/jupyter/figures/{USERNAME}/{MODEL}/{recorded_time}-images_{total_images_count}-unfreeze_{trainable_layer_count}-batch_{BATCH_SIZE}", dpi=300)

In [ ]:
model.evaluate(test_ds, callbacks=tsboard)